<a href="https://colab.research.google.com/github/ssd909/first_step_in-ml/blob/master/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
requirements = """
langchain-google-genai
langchain-huggingface
chromadb
sentence-transformers
langchain-community
pypdf
"""


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pypdf

In [5]:
from pypdf import PdfReader
file_adress = '/content/drive/MyDrive/matsne-1155567-28.pdf'
reader=PdfReader(file_adress)
number_of_pages=len(reader.pages)

In [6]:
text=""
for page in reader.pages:
  text+= page.extract_text() + "\n"
print(len(text))

118636


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)


In [8]:
chunks=text_splitter.create_documents([text])
print(len(chunks))

148


In [10]:
from google.colab import drive
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma



save_directory = '/content/drive/MyDrive/w_db'


model_name = "paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=save_directory
)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/526 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [26]:
import google.generativeai as genai

genai.configure(api_key="")




query = "რომელია საქრთველოს დედაქალაქი?"


embeddings = HuggingFaceEmbeddings(
    model_name="paraphrase-multilingual-MiniLM-L12-v2",

)
db = Chroma(persist_directory="/content/drive/MyDrive/w_db", embedding_function=embeddings)
context =  db.similarity_search(query)

prompt = f"""
გამოიყენე ქვემოთ მოცემული დოკუმენტის ნაწყვეტები კითხვაზე საპასუხოდ.
თუ პასუხი დოკუმენტში არ არის, თქვი, რომ არ იცი.


{context}

 {query}
პასუხი:"""


model = genai.GenerativeModel('gemini-2.5-flash-lite')
response = model.generate_content(prompt)
print(response.text)

არ ვიცი
